In [ ]:
#pip install -U minsearch qdrant_client


   ---------------------------------------- 337.3/337.3 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: qdrant_client
    Found existing installation: qdrant-client 1.14.2
    Uninstalling qdrant-client-1.14.2:
      Successfully uninstalled qdrant-client-1.14.2



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import requests
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
docs_url = url_prefix + 'search_evaluation/documents-with-ids.json'
documents = requests.get(docs_url).json()

ground_truth_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(ground_truth_url)
ground_truth = df_ground_truth.to_dict(orient='records')

In [11]:
from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [9]:
# Build Minsearch index
index = minsearch.Index(
    text_fields=["question", "section", "text"],
    keyword_fields=["course", "id"]
)
index.fit(documents)

boost = {'question': 1.5, 'section': 0.1}

In [10]:
# Define search function
def minsearch_search(query, course):
    return index.search(
        query=query,
        filter_dict={'course': course},
        boost_dict=boost,
        num_results=5
    )


In [12]:
# Evaluate hitrate
relevance_total = []
for q in tqdm(ground_truth):
    doc_id = q['document']
    results = minsearch_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/4627 [00:00<?, ?it/s]

### Answer:1 

In [13]:

hit = hit_rate(relevance_total)
print("Minsearch hitrate (with boost):", hit)

Minsearch hitrate (with boost): 0.848714069591528


In [16]:
%pip install -U minsearch

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: minsearch in c:\users\adi\appdata\local\programs\python\python311\lib\site-packages (0.0.4)




[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
from minsearch import VectorSearch

ImportError: cannot import name 'VectorSearch' from 'minsearch' (c:\Users\Adi\LLM-ZOOMCAMP\03-evaluation\minsearch.py)

### Answer:2

In [14]:
score = mrr(relevance_total)
print("MRR (VectorSearch, question field):", score)

MRR (VectorSearch, question field): 0.7288235717887772
